In [3]:
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import re

## Dataset creation

In [4]:
# import os
# import librosa
# import soundfile as sf

# # Assuming 'output_filename' is the desired filename for the output WAV file
# output_filename = 'long_form_audio/concatenated_output.wav'
# data=pd.read_csv('../MADHAV_task_1/Task3/metadata_test_clean.tsv', sep='\t', header=None, names=['path', 'text', 'num','sr'])
# # List to store audio chunks
# audio_chunks = []
# target=""
# # Iterate over all files in the directory
# for i in range(10):
#     file_path = f"output/audio_noise{i}.wav"

#     # Load the audio file
#     audio, sr = librosa.load(file_path, sr=None)
#     if(target==""):
#         target=data['text'][i]
#     else:
#         target+=" "+data['text'][i]
#     # Append the audio to the list
#     audio_chunks.append(audio)

# # Concatenate all audio chunks
# concatenated_audio = np.concatenate(audio_chunks)

# # Save the concatenated audio to a new WAV file
# sf.write(output_filename, concatenated_audio, samplerate=sr)
# print(target)

In [5]:
# %cd /home/suryansh/MADHAV

In [6]:
# txt_file_path = "./hindi_data/givenFolderHSp/txt/12janA.txt"
# file=open("./hindi_data/givenFolderHSp/txt/12janA.txt", 'r')
# target=file.readlines()
# target=re.sub(" +", " ", " ".join(target))
# # Now 'hindi_text' contains the contents of the file as a string
# print(target)

## Model algorithm

In [7]:
def split_audio_using_VAD(input_file, pred, chunk_length_sec=15):
    # Load the audio file
    audio, sr = librosa.load(input_file, sr=None)
    timestamps_df=pd.DataFrame()
    timestamps_df['start']=pred['start'].apply(lambda x: x*sr/1000)
    timestamps_df['end']=pred['end'].apply(lambda x: x*sr/1000)
    print(timestamps_df)
    # Calculate the total duration of the audio in samples
    total_samples = len(audio)

    # Calculate the chunk length in samples
    chunk_length_samples = int(sr * chunk_length_sec)
    print(chunk_length_samples)
    # Create an array to store audio chunks
    audio_chunks = []
    start_sample=0
    end_sample=chunk_length_samples
    split_sec=[]
    k=0
    index=0
    for i, row in timestamps_df.iterrows():
        if (start_sample+chunk_length_samples)>=row['end']:
            end_sample=row['end']
        else:
            audio_chunks.append(audio[int(start_sample):int(end_sample)])
            split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
            index+=1
            start_sample=end_sample
            end_sample=row['end']
            while(start_sample+chunk_length_samples<row['end']):
                end_sample=start_sample+chunk_length_samples
                audio_chunks.append(audio[int(start_sample):int(end_sample)])
                split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
                start_sample=end_sample
                index+=1
                end_sample=row['end']
            

    last_chunk = audio[int(start_sample):int(total_samples)]
    audio_chunks.append(last_chunk)
    split_sec.append((index, start_sample/sr, total_samples/sr))
    split_sec=pd.DataFrame(split_sec, columns=['index', 'start', 'end'])
    return audio_chunks, split_sec

In [8]:
from panns_inference import AudioTagging, SoundEventDetection, labels
import time
def time_cost(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        ret = func(*args, **kwargs)
        time_cost = time.time() - start
        return ret
    return wrapper
class SoundEventDetectionX(SoundEventDetection):
    @time_cost
    def detect_sound_event(self, audio_path, n_dominate=5, threshold=0.2):
        """Save sound event detection result as CSV. 

        Args:
          framewise_output (np.array): (time_steps, classes_num)
          n_dominate (int): number of dominated events
          threshold (float): threshold to determine whether the events occur.

        Returns:
          time_stamps (list): a list of sound event time-stamps ({'event', 'id', 'start', 'stop'})
        """
        (audio, _) = librosa.core.load(audio_path, sr=32000, mono=True)
        duration = audio.shape[0]/32000
        audio = audio[None, :]  # (batch_size, segment_samples)

        framewise_output = self.inference(audio) # (batch_size, time_steps, classes_num)
        framewise_output = framewise_output[0]

        classwise_output = np.max(framewise_output, axis=0) # (classes_num,)
        dominate_events = np.argsort(classwise_output)[::-1]
        dominate_events = dominate_events[0:n_dominate]

        ix_to_lb = {i : label for i, label in enumerate(labels)}
        
        time_stamps = []
        for event_id in dominate_events:
            event_info = dict()
            event_label = ix_to_lb[event_id]
            event_info["prob"] = framewise_output[:, event_id]
            activations = framewise_output[:, event_id] > threshold
            event_info["active"] = activations

            capture_event = False
            start_idx = 0
            for time_idx, activate in enumerate(activations):
                if capture_event:
                    if activate == False:
                        capture_event = False
                        time_stamps.append({"label": event_label, "start": round(0.01*start_idx, 4), "stop": round(0.01*time_idx, 4)})
                    elif time_idx == len(activations)-1:
                        time_stamps.append({"label": event_label, "start": round(0.01*start_idx, 4), "stop": round(0.01*time_idx, 4)})
                else:
                    if activate == True:
                        capture_event = True
                        start_idx = time_idx

        self.sort_timestamps(time_stamps, 0, len(time_stamps)-1)

        for ind, time_stamp in enumerate(time_stamps):
            time_stamp["id"] = ind
            if time_stamp["stop"] < duration:
                # Avoid cutting word tail
                time_stamp["stop"] += 0.5
        return time_stamps

    def sort_timestamps(self, time_stamps, head_index, tail_index):
        """ Sort time-stamps based on start time. 
        
        Use quick-sort and pivot is always the last.
        """
        if tail_index >= len(time_stamps):
            raise ValueError("tail_index must less or equal to length of timestamps-1.")

        if tail_index > head_index:
            pivot_index = tail_index
            counter = 0
            for index in range(head_index, tail_index):
                if time_stamps[index-counter]["start"] > time_stamps[pivot_index]["start"]:
                    bigger = time_stamps.pop(index-counter)
                    time_stamps.insert(tail_index, bigger)
                    pivot_index -= 1
                    counter += 1
            self.sort_timestamps(time_stamps, head_index, pivot_index-1)
            self.sort_timestamps(time_stamps, pivot_index+1, tail_index)
        return time_stamps

In [9]:
# model_path = "~/panns_data/Cnn14_DecisionLevelMax.pth"
# input_file = "./hindi_data/givenFolderHSp/fullAudio/12janA.wav"
# sed = SoundEventDetectionX(checkpoint_path=model_path, device='cuda')
# event_timestamps = sed.detect_sound_event(input_file)
# event_timestamps
# df=pd.DataFrame(event_timestamps)
# df['end']=df['stop']
# df.drop(['stop', 'id', 'label'], axis=1, inplace=True)
# df=df*1000

In [10]:
# # Split the audio into 15-second chunks with adjustment for the last chunk
# chunks_array, split_sec = split_audio_using_VAD(input_file, df, 7)

# # Now, chunks_array contains the audio chunks as numpy arrays
# for i, chunk in enumerate(chunks_array):
#     print(f"Chunk {i}: start: {split_sec['start'][i]} end: {split_sec['end'][i]}")

In [11]:
# !pip install -q espnet_model_zoo

In [12]:
# #@title Choose English ASR model { run: "auto" }

# lang = 'en'
# fs = 16000 #@param {type:"integer"}
# tag = 'viks66/asr_train_asr_raw_hindi_bpe500' #@param ["Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave", "kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave"] {type:"string"}

In [13]:
# !git lfs install
# !git clone https://huggingface.co/viks66/asr_train_asr_raw_hindi_bpe500

In [14]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [15]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [16]:
# import pandas as pd
# import librosa.display
# from IPython.display import display, Audio
# import matplotlib.pyplot as plt
# import time
# import numpy as np
# import soundfile as sf

# col_names = ['path', 'text', 'time', 'sr']
# preds = ""
# target = ""

# for i, chunk in enumerate(chunks_array):
#     start_time = time.time()
#     speech = np.array([])
#     duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
#     for _, row in df.iterrows():
#         start_sample = row['start']-split_sec['start'][i] * 1000
#         end_sample = row['end']-split_sec['start'][i] * 1000
# #         print(start_sample, end_sample, duration)
#         if(start_sample<0 and end_sample<0):
#             continue
#         if(start_sample>duration and end_sample>duration):
#             break
# #         print("Y")
#         speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

#     print(len(speech))
# #     nbests = speech2text(chunk)
#     if len(speech) != 0:
#         nbests = speech2text(speech)
#         text, *_ = nbests[0]
# #         output_filename = f'results/output_chunk{i}.wav'
# #         sf.write(output_filename, speech, samplerate=16000)
#         if(preds==""):
#             preds=(text_normalizer(text))
#         else:
#             preds += " " + (text_normalizer(text))

#     print(i, "/500")
#     elapsed_time = time.time() - start_time
#     print(f"Time taken: {elapsed_time:.2f} seconds")


In [17]:
predictions=[]
ground_truth=[]

In [20]:
import pandas as pd
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time
import numpy as np
import soundfile as sf
import torch
# files=['./hindi_data/givenFolderHSp/fullAudio/12janA.wav', './hindi_data/givenFolderHSp/fullAudio/11janA.wav','./hindi_data/givenFolderPBData/fullAudio/8483_FullAudio.wav','./hindi_data/givenFolderPBData/fullAudio/17162_FullAudio.wav','./hindi_data/givenPMS/newGivenPMS/longAudio/april_v.wav' ]
# txts=['./hindi_data/givenFolderHSp/txt/12janA.txt', './hindi_data/givenFolderHSp/txt/11janA.txt','./hindi_data/givenFolderPBData/txt/8483_FullAudio.txt','./hindi_data/givenFolderPBData/txt/17162_FullAudio.txt','./hindi_data/givenPMS/newGivenPMS/txt/april_v.txt']
files=['./hindi_data/givenFolderPBData/fullAudio/17162_FullAudio.wav']
txts=['./hindi_data/givenFolderPBData/txt/17162_FullAudio.txt']
for txt_file_path, input_file in zip(txts, files):
    %cd /home/suryansh/MADHAV
    file=open(txt_file_path, 'r')
    target=""
    target=file.readlines()
    target=re.sub(" +", " ", " ".join(target))
    # Now 'hindi_text' contains the contents of the file as a string
    print(target)
    ground_truth.append(target)
#     print(ground_truth)
    model_path = "~/panns_data/Cnn14_DecisionLevelMax.pth"
    sed = SoundEventDetectionX(checkpoint_path=model_path, device='cuda')
    event_timestamps = sed.detect_sound_event(input_file)
    event_timestamps
    df=pd.DataFrame(event_timestamps)
    df['end']=df['stop']
    df.drop(['stop', 'id', 'label'], axis=1, inplace=True)
    df=df*1000
    # Split the audio into 15-second chunks with adjustment for the last chunk
    chunks_array, split_sec = split_audio_using_VAD(input_file, df, 7)
    for i, chunk in enumerate(chunks_array):
        print(f"Chunk {i}: start: {split_sec['start'][i]} end: {split_sec['end'][i]}")
    %cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500
    print()
    preds = ""
    for i, chunk in enumerate(chunks_array):
        start_time = time.time()
        speech = np.array([])
        duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
        for _, row in df.iterrows():
            start_sample = row['start']-split_sec['start'][i] * 1000
            end_sample = row['end']-split_sec['start'][i] * 1000
    #         print(start_sample, end_sample, duration)
            if(start_sample<0 and end_sample<0):
                continue
            if(start_sample>duration and end_sample>duration):
                break
    #         print("Y")
            speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

        print(len(speech))
    #     nbests = speech2text(chunk)
        if len(speech) != 0:
            nbests = speech2text(speech)
            text, *_ = nbests[0]
    #         output_filename = f'results/output_chunk{i}.wav'
    #         sf.write(output_filename, speech, samplerate=16000)
            if(preds==""):
                preds=(text_normalizer(text))
            else:
                preds += " " + (text_normalizer(text))

        print(i, "/", len(chunks_array))
        elapsed_time = time.time() - start_time
        print(f"Time taken: {elapsed_time:.2f} seconds")
    predictions.append(preds)
    torch.cuda.empty_cache()    

/home/suryansh/MADHAV
 श्री लाल जी आपने हिंदी की तमाम विधाओं में काम किया है उपन्यास कहानी निबंध आलोचना और शुरुआत में कविताएं भी लिखी आपने तो इतनी विधवाओं में सक्रिय होने के बावजूद अगर मैं आपसे पूछूं कि आप मूल रूप से किस विधा के लेखक है वास्तव मे स्थिति ये है की चालीस साल से जब कभी भी कोई रचनात्मक कार्य उसके विषय में सोचना शुरू किया है तो मूर्धा तो कथा के एक क्षेत्र में है सोचा है वो कथा के भी क्षेत्र में अगर कहीं किसी कहानी की भी अवधारणा दिमाग में आई तो उस पर ज्यादा सोचते सोचते फिर ऐसा लगता है कि वो अपने आप ही एक उपन्यास का रूप ले तो अगर इस सहज वृद्धि के आधार पर चला जाये तो मुझे ऐसा लगता है की वास्तविक रुझान मेरा उपन्यास की ओर है और शायद यही कारण था कि जब मैंने लिखना आरंभ किया है तो मेरी पहली पुस्तक उपन्यास ही हैं अच्छा लेकिन जब आपका जो ये गद्य लेखन का दौर जब शुरू होता है पचास के आस पास तो आप देखिये की यही समय था हिंदी में नए स्वर्ण युग का है उसकी भी सक्रियता का यही दौर था और आपके समकालीन जो अन्य लेखक थे वे सब एक प्रमुखता कहानियाँ लिख रहे थे तो ऐसी क्या बात थी कि आपको लगा की नहीं कहान

OutOfMemoryError: CUDA out of memory. Tried to allocate 144.00 MiB. GPU 0 has a total capacty of 7.75 GiB of which 133.38 MiB is free. Including non-PyTorch memory, this process has 7.61 GiB memory in use. Of the allocated memory 7.45 GiB is allocated by PyTorch, and 2.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
# print([ground_truth[i] for i in [0,3,4]])
print(ground_truth[:-1])

[' दोपहर समाचार के साथ मैं निखिल कुमार प्रधानमंत्री नरेंद्र मोदी ने कहा है कि ग्लोबल साउथ को विकास की धारा से अलग नही रखा जा सकता आज वर्चुअल माध्यम से आयोजित वायस ऑफ ग्लोबल साउथ समिट के उद्घाटन सत्र में श्री मोदी ने कहा कि ग्लोबल साउथ यानी विश्व के कम विकसित और विकासशील देशों की भविष्य में सबसे बडी भागीदारी है उन्होंने कहा कि दुनिया की तीन चौथाई जनसख्ंया इन्हीं देशों में रहती है और उन आवाज को भी समान महत्व दिया जाना चाहिए प्रधानमंत्री ने कहा कि अधिकतर चुनौतियां कम विकसित देशों के कारण नहीं हुई है लेकिन इनका प्रभाव इन्हीं देशों पर पड़ता है मोस्ट ऑफ प्रधानमंत्री ने कहा कि भारत ने हमेशा अपने विकास से जुडे अनुभवों को विकासशील देशों के साथ साझा किया श्री मोदी ने विश्वास व्यक्त किया कि ग्लोबल साउथ के देश मिलकर नए और रचनात्मक विचारों का सृजन कर सकते हैं और ये विचार जी ट्वेंटी तथा अन्य मंचों पर हमारी आवाज का आधार बन सकते हैं श्री मोदी ने कहा कि जी ट्वेंटी की भारत की अध्यक्षता के दौरान एक पृथ्वी एक परिवार एक भविष्य का सिद्धान्त अपनाया गया है जो मानव केंद्रित विकास के माध्यम से एकात्मकता का सृजन 

In [18]:
print(predictions)

['दोपहर तमाचार के साथ में अनिखिल कुमार प्रधानमंत्री नरेंद्र मोदी ने कहा है कि ग्लोबल साउथ प्रधानमंत्री नरेंद्र मोदी ने कहा प्रधानमंत्री नरेंद्र मोदी ने कहा को विकास की धारा से अलग नहीं रखा जा सकता आज वह शुर्माध्यम से आयोजित वाइपोफ ग्लोबल साउथ पर उद्घाटन सत्र में उन्होंने कहा कि ग्लोबल साउथ यानी विश्व के कम विकसित और विकासशील देशों की भविष्य में सत बड़ी भागीदारी है उन्होंने कहा कि दुनिया की तीन चौथाई जनसंख्या इन्हीं देशों में रहती है और उनकी आवाज को भी प्रमान महत्व दिया जाना चाहिए प्रधानमंत्र ने कहा कि अधिक पर चुनौतियां कम विकसित प्रदेशों के कारण नहीं हुई हैं लेकिन इनका प्रभाव इन्हीं देशों पर पड़ता है वह शब्द प्रधानमंथण ने कहा कि भारत ने हमें इशा अपने विकास से जुड़े अनुभवों को विकास सुशील देशों के साथ साठा किया है कि मोदी ने विश्वास से किया कि लोबल साउथ के देश मिलकर नए और रचनात्मक विचारों का स्ट्रेजन कर सकते हैं और विचार जी गैंटी तथा नमजों पर हमारी आवाज का आधार बन सकते हैं किमोद ने कहा कि जीपेंटी की भारत की अध्यक्षता के दौरान एक पृथ्वी एक परिवार एक भविष्य का सिद्धांत अपनाया गया है जो मा

In [20]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions, ground_truth[:-1]))
wer = WordErrorRate()
print("WER:", wer(predictions, ground_truth[:-1]))

CER: tensor(0.2854)
WER: tensor(0.4647)


In [82]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [83]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [84]:
import pandas as pd
import soundfile
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time

col_names=['path', 'text', 'time', 'sr', 'std']
egs = pd.read_csv("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/tsv/12janA.tsv", sep="\t", names=col_names, header=None, encoding='utf-8', index_col=False)
# print(egs['path'])
preds=[]
target=[]
for index, row in egs.iterrows():
    start_time = time.time()
    speech, rate = soundfile.read("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/finalSplitAudio/" + row["path"])
#     assert fs == int(row["sr"])
    nbests = speech2text(speech)

    text, *_ = nbests[0]
# Uncomment to see details of each prediction
#     print(f"Input Speech: ESPNet_asr_egs/{row['path']}")
    # let us listen to samples
#     display(Audio(speech, rate=rate))
#     librosa.display.waveshow(speech, sr=rate, color="blue")
#     plt.show()
#     print(f"Reference text: {text_normalizer(row['text'])}")
    target.append(text_normalizer(row['text']))
#     print(f"ASR hypothesis: {text_normalizer(text)}")
    preds.append(text_normalizer(text))
    print(index,"/500")
    elapsed_time = time.time() - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 /500
Time taken: 5.14 seconds
1 /500
Time taken: 5.37 seconds
2 /500
Time taken: 4.41 seconds
3 /500
Time taken: 2.48 seconds
4 /500
Time taken: 4.52 seconds
5 /500
Time taken: 4.49 seconds
6 /500
Time taken: 5.41 seconds
7 /500
Time taken: 5.35 seconds
8 /500
Time taken: 5.75 seconds
9 /500
Time taken: 5.21 seconds
10 /500
Time taken: 4.13 seconds
11 /500
Time taken: 5.15 seconds
12 /500
Time taken: 6.05 seconds
13 /500
Time taken: 5.67 seconds
14 /500
Time taken: 6.45 seconds
15 /500
Time taken: 5.18 seconds
16 /500
Time taken: 5.57 seconds
17 /500
Time taken: 5.97 seconds
18 /500
Time taken: 2.56 seconds
19 /500
Time taken: 5.08 seconds
20 /500
Time taken: 5.53 seconds
21 /500
Time taken: 2.00 seconds
22 /500
Time taken: 4.25 seconds
23 /500
Time taken: 5.51 seconds
24 /500
Time taken: 6.48 seconds
25 /500
Time taken: 5.60 seconds
26 /500
Time taken: 4.99 seconds
27 /500
Time taken: 5.11 seconds
28 /500
Time taken: 5.03 seconds
29 /500
Time taken: 5.10 seconds
30 /500
Time taken: 

In [85]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(preds, target))
wer = WordErrorRate()
print("WER:", wer(preds, target))

CER: tensor(0.1677)
WER: tensor(0.3445)
